In [ ]:
# TODO: Ver 05_qaoa_qiskit_tutorial.ipynb para hacer con qiskit.algorithms.QAOA

from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import SparsePauliOp
from qiskit.visualization import plot_distribution
from qiskit.primitives import Estimator, Sampler

from scipy.optimize import minimize

from typing import Callable, Dict, List, Tuple

In [ ]:
# Primer grafo
# P = 27
GRAPH_NAME = "Primer grafo"

NUM_NODES = 4
EDGES = {(0, 1): 5, (0, 2): 8, (1, 2): 2, (1, 3): 7, (2, 3): 4}
NQUBITS = len(EDGES)

# Hp = 11*Z0 - 17.5*Z1 - 28*Z2 - 17*Z3 + 11.5*Z4 +
#      13.5*(-Z0*Z2 + Z1*Z2 + Z2*Z3 - Z2*Z4 + Z0*Z1 - Z0*Z3 - Z1*Z4)

# Coeficientes lineales de Hp (11 * Z0 - 17.5 * Z1...)
LINEAR_COEFS = [11, -17.5, -28, -17, 11.5]

# Coeficientes cuadráticos Ej: {(2, 3): 13.5} -> 13.5 * Z2 * Z3
QUADRA_COEFS = {(0, 1): 13.5, (0, 2): -13.5, (0, 3): -13.5, (1, 2): 13.5,
                (1, 4): -13.5, (2, 3): 13.5, (2, 4): -13.5}

PRINT_GRAPH_LAYOUT = {0: [-1, 0], 1: [0, 1], 2: [0, -1], 3: [1, 0]}

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Imprime un grafo dirigido pesado
# Entrada:
#   - graph_nodes: Lista de nodos del grafo
#   - graph_edges: Diccionario {edge: weight} donde edge es (n1, n2)
#   - layout: Para especificar la posición de los nodos
def print_graph(graph_nodes: List[int],
                graph_edges: Dict[Tuple[int, int], int],
                title="", layout: Dict[int, List[int]]=None):
    G = nx.DiGraph()

    # Nodes
    G.add_nodes_from(graph_nodes)

    # Edges
    for pair, weight in graph_edges.items():
        G.add_edge(pair[0], pair[1], weight=weight)

    # Print graph
    if layout is None:
        layout = nx.spring_layout(G)

    plt.figure(3, figsize=(6, 3))
    plt.title(title)
    nx.draw(G, layout, with_labels=True, node_size=600)
    edge_labels = nx.get_edge_attributes(G, "weight")
    nx.draw_networkx_edge_labels(G, pos=layout, edge_labels=edge_labels)
    #plt.savefig("../../latex/img/primer_grafo/primer_grafo.png")


print_graph(range(NUM_NODES), EDGES, title=GRAPH_NAME, layout=PRINT_GRAPH_LAYOUT)

In [ ]:
# Create list for SparsePauliOp
hamiltonian_list = []
for coef, qubit_idx in zip(LINEAR_COEFS, range(len(LINEAR_COEFS))):
    gates = ["I"] * NQUBITS
    gates[(NQUBITS - qubit_idx - 1)] = "Z"
    hamiltonian_list.append(("".join(gates), coef))

for qubit_idxs in QUADRA_COEFS:
    coef = QUADRA_COEFS[qubit_idxs]
    gates = ["I"] * NQUBITS
    gates[(NQUBITS - qubit_idxs[0] - 1)] = "Z"
    gates[(NQUBITS - qubit_idxs[1] - 1)] = "Z"
    hamiltonian_list.append(("".join(gates), coef))

hamiltonian_list

In [ ]:
hamiltonian = SparsePauliOp.from_list(hamiltonian_list)
num_layers = 1
ansatz = QAOAAnsatz(hamiltonian, reps=num_layers)
ansatz.decompose(reps=3).draw(output="mpl")

In [ ]:
def cost_func(params, ansatz, hamiltonian, estimator):
    cost = estimator.run(ansatz, hamiltonian, parameter_values=params).result().values[0]
    return cost

In [ ]:
estimator = Estimator(options={"shots": int(1000)})
sampler = Sampler(options={"shots": int(1000)})

In [ ]:
x0 = [1.0, 1.0] * num_layers
res = minimize(cost_func, x0, args=(ansatz, hamiltonian, estimator), method="COBYLA")
res

In [ ]:
qc = ansatz.assign_parameters(res.x)
qc.measure_all()
samp_dist = sampler.run(qc).result().quasi_dists[0]

In [ ]:
plot_distribution(samp_dist.binary_probabilities())

In [ ]:
# Statistics
def max_global_statistics(num_layers=1,
                          num_generations=100) -> Tuple[Dict[str, float],
                                                        Dict[str, float]]:
    max_statistics = {}
    global_statistics = {}
    ansatz = QAOAAnsatz(hamiltonian, reps=num_layers)
    x0 = [1.0, 1.0] * num_layers
    for iteration in range(0, num_generations):
        theta_res = minimize(cost_func, x0, args=(ansatz, hamiltonian, estimator), method="COBYLA")
        qc = ansatz.assign_parameters(theta_res.x)
        qc.measure_all()
        counts = sampler.run(qc).result().quasi_dists[0].binary_probabilities()

        # Max statistics
        max_path = max(counts, key=counts.get)
        if max_path not in max_statistics:
            max_statistics[max_path] = 0
        max_statistics[max_path] += 1

        # Global statistics
        for (path, num_appearances) in counts.items():
            if path not in global_statistics:
                global_statistics[path] = 0
            global_statistics[path] += num_appearances

    for path in global_statistics:
        global_statistics[path] = global_statistics[path] / num_generations  # Normalize

    for path in max_statistics:
        max_statistics[path] = max_statistics[path] / num_generations  # Normalize

    max_statistics = sorted(max_statistics.items(), key=lambda x: x[1], reverse=True)
    global_statistics = sorted(global_statistics.items(), key=lambda x: x[1], reverse=True)
    return max_statistics, global_statistics

print(GRAPH_NAME + ":\n")
interval_num_layers = (1, 5)
for p in range(interval_num_layers[0], interval_num_layers[1] + 1):
    max_st, global_st = max_global_statistics(num_layers=p, num_generations=100)
    print(f"Max statistics (p = {p}): ", str(max_st))
    print(f"Global statistics (p = {p}): ", str(global_st))
    print()

In [ ]:
Primer grafo:

Max statistics (p = 1):  [('10101', 0.01)]
Global statistics (p = 1):  [('10101', 0.02138999999999997)]

Max statistics (p = 2):  [('10101', 0.06)]
Global statistics (p = 2):  [('10101', 0.04259999999999997)]

Max statistics (p = 3):  [('10101', 0.52)]
Global statistics (p = 3):  [('10101', 0.12084999999999996)]

Max statistics (p = 4):  [('10101', 0.05281999999999999)]

Max statistics (p = 5):  [('10101', 0.07)]
Global statistics (p = 5):  [('10101', 0.06544999999999998)]

In [ ]:
Primer grafo:

Max statistics (p = 1):  [('00000', 0.78), ('00010', 0.07), ('10001', 0.05), ('01001', 0.04), ('10010', 0.02), ('10000', 0.02), ('10101', 0.01), ('01000', 0.01)]
Global statistics (p = 1):  [('00000', 0.27853), ('00010', 0.15205), ('00001', 0.12320000000000003), ('00011', 0.08400999999999997), ('01000', 0.049309999999999944), ('00100', 0.034869999999999984), ('10001', 0.034429999999999975), ('10000', 0.03420999999999997), ('00110', 0.032129999999999985), ('10101', 0.02138999999999997), ('01001', 0.021109999999999983), ('10010', 0.020249999999999987), ('00101', 0.01761999999999999), ('01010', 0.014249999999999994), ('00111', 0.012679999999999995), ('11000', 0.01146), ('01011', 0.006730000000000003), ('01100', 0.006630000000000004), ('01101', 0.005440000000000001), ('10100', 0.0052600000000000025), ('10011', 0.004920000000000003), ('11001', 0.004760000000000001), ('01110', 0.004500000000000003), ('11101', 0.00394), ('11010', 0.0033900000000000007), ('10111', 0.0033500000000000005), ('10110', 0.0030200000000000014), ('01111', 0.0021200000000000012), ('11100', 0.0019500000000000003), ('11011', 0.0012700000000000003), ('11111', 0.00072), ('11110', 0.0005000000000000001)]

Max statistics (p = 2):  [('01001', 0.44), ('10010', 0.1), ('00001', 0.07), ('10101', 0.06), ('01000', 0.06), ('10001', 0.05), ('10000', 0.04), ('11001', 0.03), ('11011', 0.03), ('00000', 0.03), ('11010', 0.03), ('00010', 0.02), ('01010', 0.01), ('10111', 0.01), ('00101', 0.01), ('11101', 0.01)]
Global statistics (p = 2):  [('01001', 0.21896000000000002), ('00001', 0.07266999999999997), ('11001', 0.056489999999999964), ('00000', 0.05462000000000001), ('01000', 0.04950999999999997), ('10010', 0.046709999999999974), ('10101', 0.04259999999999997), ('10001', 0.04189999999999998), ('10000', 0.03966999999999999), ('01011', 0.037839999999999985), ('00101', 0.029009999999999984), ('00010', 0.028649999999999988), ('01101', 0.025239999999999974), ('11011', 0.024399999999999977), ('11101', 0.02335999999999998), ('11010', 0.02283999999999999), ('10100', 0.021419999999999984), ('01010', 0.02117999999999998), ('11000', 0.019569999999999987), ('00100', 0.01722999999999999), ('00011', 0.015909999999999994), ('10011', 0.01561999999999999), ('10111', 0.013039999999999994), ('10110', 0.010430000000000002), ('01111', 0.009780000000000006), ('01100', 0.007550000000000005), ('11100', 0.007200000000000004), ('11111', 0.007080000000000003), ('00111', 0.005790000000000003), ('00110', 0.005570000000000003), ('01110', 0.004290000000000002), ('11110', 0.003870000000000002)]

Max statistics (p = 3):  [('10101', 0.52), ('00001', 0.25), ('00000', 0.15), ('11010', 0.06), ('10110', 0.01), ('10001', 0.01)]
Global statistics (p = 3):  [('10101', 0.12084999999999996), ('00001', 0.08998999999999999), ('00000', 0.08719999999999996), ('10010', 0.05759), ('01101', 0.050550000000000025), ('11101', 0.04351999999999995), ('01001', 0.041670000000000006), ('10000', 0.039249999999999986), ('10001', 0.038469999999999976), ('10100', 0.03710999999999998), ('11010', 0.036219999999999974), ('10111', 0.032039999999999985), ('00100', 0.029419999999999988), ('01000', 0.029069999999999988), ('11000', 0.02631999999999999), ('00010', 0.023089999999999992), ('00101', 0.0226), ('00011', 0.022449999999999984), ('11001', 0.019679999999999986), ('01011', 0.018869999999999994), ('10110', 0.01714999999999999), ('11111', 0.01521), ('11100', 0.014359999999999993), ('10011', 0.013460000000000001), ('11110', 0.012140000000000001), ('01010', 0.011949999999999994), ('01100', 0.010450000000000006), ('00110', 0.010020000000000005), ('01111', 0.009570000000000006), ('11011', 0.0073600000000000046), ('00111', 0.006570000000000003), ('01110', 0.005800000000000004)]

Max statistics (p = 4):  [('10010', 0.2), ('01001', 0.17), ('00000', 0.16), ('10000', 0.09), ('10101', 0.07), ('00001', 0.06), ('10100', 0.05), ('01011', 0.02), ('11101', 0.02), ('00101', 0.02), ('00010', 0.02), ('01000', 0.02), ('01010', 0.02), ('10111', 0.02), ('10001', 0.02), ('10011', 0.01), ('11001', 0.01), ('11000', 0.01), ('11011', 0.01)]
Global statistics (p = 4):  [('01001', 0.08922999999999998), ('10010', 0.08211999999999998), ('00000', 0.07476), ('10000', 0.060309999999999996), ('10101', 0.05281999999999999), ('00001', 0.04951999999999997), ('10100', 0.041479999999999996), ('01000', 0.04049999999999998), ('00101', 0.04037999999999998), ('11001', 0.033439999999999984), ('00010', 0.03297999999999998), ('01010', 0.03269999999999998), ('10001', 0.03256), ('11101', 0.030669999999999985), ('10111', 0.02607999999999998), ('01101', 0.025469999999999993), ('11011', 0.025429999999999994), ('01011', 0.02280999999999998), ('10110', 0.022699999999999988), ('11010', 0.022359999999999998), ('11000', 0.02028999999999999), ('00011', 0.01944999999999998), ('10011', 0.019039999999999984), ('00100', 0.018749999999999985), ('00110', 0.012419999999999994), ('01111', 0.011989999999999999), ('11111', 0.0113), ('11100', 0.011280000000000005), ('11110', 0.010950000000000001), ('01100', 0.009830000000000005), ('00111', 0.009570000000000006), ('01110', 0.0068100000000000036)]

Max statistics (p = 5):  [('01001', 0.2), ('00101', 0.17), ('10000', 0.15), ('10010', 0.12), ('00001', 0.1), ('00010', 0.09), ('10101', 0.07), ('10100', 0.07), ('11001', 0.01), ('11010', 0.01), ('10001', 0.01)]
Global statistics (p = 5):  [('01001', 0.10120999999999994), ('00001', 0.09979999999999999), ('00101', 0.09365), ('10101', 0.06544999999999998), ('10010', 0.05595999999999998), ('00010', 0.051709999999999985), ('10100', 0.04900999999999999), ('10000', 0.04820999999999999), ('11001', 0.04573999999999996), ('10001', 0.03794999999999998), ('11011', 0.03713999999999998), ('00000', 0.026169999999999978), ('11101', 0.024959999999999986), ('01011', 0.024889999999999985), ('00100', 0.02227999999999998), ('11010', 0.021989999999999975), ('10110', 0.02023999999999999), ('10111', 0.019869999999999985), ('01000', 0.018709999999999984), ('01101', 0.01623999999999999), ('11000', 0.01581999999999999), ('11100', 0.014919999999999996), ('01010', 0.014029999999999994), ('00011', 0.010810000000000004), ('01111', 0.010730000000000003), ('10011', 0.010730000000000002), ('00111', 0.010720000000000007), ('01110', 0.007930000000000005), ('01100', 0.007600000000000004), ('00110', 0.006020000000000003), ('11110', 0.004770000000000002), ('11111', 0.004740000000000003)]

In [ ]:
!dunstify -t $((12 * 60*60*1000)) --urgency=critical "Ejecución completada"